In [2]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
#import pinecone
from pinecone import Pinecone
from langchain.llms import CTransformers

#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

index_name="medical-chatbot"
PINECONE_API_KEY = "75958ab5-04b8-452b-b085-4101fd324846"
PINECONE_API_ENV = "gcp-starter"
pc = Pinecone(api_key=PINECONE_API_KEY)
embeddings = download_hugging_face_embeddings()
os.environ['PINECONE_API_KEY']= PINECONE_API_KEY
docsearch = PineconeStore.from_existing_index(index_name=index_name, embedding=embeddings)

c:\Users\npime\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [4]:
from langchain.prompts import PromptTemplate
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}


In [5]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [6]:
from langchain.chains import RetrievalQA
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [7]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\npime\anaconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  The symptoms of major depression can be treated with a combination of psychotherapy, medication, and lifestyle changes. Cognitive-behavioral therapy (CBT) is a type of psychotherapy that can help individuals identify and change negative thought patterns and behaviors that contribute to their depression. Antidepressant medications, such as selective serotonin reuptake inhibitors (SSRIs), can also be effective in treating major depression. Lifestyle changes, such as regular exercise, healthy eating, and getting enough sleep, can also help manage symptoms of depression. It's important to seek professional help if you or someone you know is experiencing symptoms of major depression.
Response :  Based on the information provided, it seems that you may have come down with a common cold or flu. These illnesses can cause similar symptoms such as headache, body aches, sore throat, and mild fever. It's important to rest, stay hydrated, and manage your symptoms through over-the-counte

KeyboardInterrupt: 